In [ ]:
# You might need to install spacy and download an english model
# !pip install spacy
# !python -m spacy download en

In [1]:
# We have a bunch of emails on disk

import glob
import io
import spacy
from collections import Counter
import pandas as pd
import email
import re

# nlp = spacy.load('en')

# corpus = Counter()
# emails = []

# with open('./data/SPAMTrain.label', 'r') as file:
#     for i, line in enumerate(file):
#         # You can always look at all of the files too!
#         print('.', end='')
        
#         spam, eml = line.split(' ')
#         eml = "./data/TRAINING/{}".format(eml.strip())
        
#         c = Counter()
#         with open(eml, 'r', errors='replace') as full_message: 
#             msg = email.message_from_string(full_message.read())
#             if not msg.is_multipart() and msg.get_content_type() == 'text/plain':
#                 payload = str(msg.get_payload())
#                 for t in nlp(re.sub(r'\s+', ' ', payload)):
#                     c[t.lemma_] += 1

#         if len(c) > 0:
#             emails.append({'SPAM_VALUE': spam, **dict(c)})

# eml_df = pd.DataFrame(emails)

# keys = []

# for c in eml_df.columns:
#     if eml_df[c].sum() > 10:
#         print('.', end='')
#         keys.append(c)

# eml_df = eml_df[keys]

# eml_df.to_csv('./email-training.csv')

In [6]:
eml_df = pd.read_csv('./email-training.csv')

In [7]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report

clf = GaussianNB()

X = eml_df[eml_df.columns[~eml_df.columns.isin(['SPAM_VALUE'])]].fillna(value=0)
y = eml_df['SPAM_VALUE']

clf.fit(X, y)

# Why is the value what it is? What would you do to fix this?

print(classification_report(clf.predict(X), y))

             precision    recall  f1-score   support

          0       1.00      0.58      0.73       643
          1       0.89      1.00      0.94      2298

avg / total       0.92      0.91      0.90      2941



In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = GaussianNB()

clf.fit(X_train, y_train)

print(classification_report(clf.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.94      0.64      0.76        96
          1       0.93      0.99      0.96       493

avg / total       0.93      0.93      0.93       589



In [35]:
from sklearn.metrics import confusion_matrix

confusion_matrix(clf.predict(X_test), y_test)

array([[ 38,  16],
       [ 17, 398]])

I was able to get ~0.94 for precision on both Ham(1) and Spam(0). That's pretty good! The recall isn't so good actually. A 0.60 recall isn't so great, although we could probably do better on parsing the Emails themselves. Think of some reasons how this result could get better.

# BONUS word2vec

Word2Vec is an amazing model to fit natural language to a vectorized representation. Matter of fact it's called representation learning and basically will take text and output something that we can do math on!

Below I've done the heavy lifting of giving you a word2vec file to train on. Try out GaussianNB and see what happens. Is it any better or worse?

In [22]:
# emails = []
# import numpy as np

# with open('./data/SPAMTrain.label', 'r') as file:
#     for i, line in enumerate(file):
#         # You can always look at all of the files too!
#         print('.', end='')
        
#         spam, eml = line.split(' ')
#         eml = "./data/TRAINING/{}".format(eml.strip())
        
#         vector = np.zeros(0)
#         with open(eml, 'r', errors='replace') as full_message: 
#             msg = email.message_from_string(full_message.read())
#             if not msg.is_multipart() and msg.get_content_type() == 'text/plain':
#                 payload = str(msg.get_payload())
                
#                 s = nlp(re.sub(r'\s+', ' ', payload))
                
#                 vector = s.vector

#         if np.sum(vector) > 0:
#             emails.append({'SPAM_VALUE': spam, **dict(zip(range(384), vector))})

# eml_df = pd.DataFrame(emails)

# eml_df.to_csv('./emails-word2vec.csv')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [23]:
eml_df = pd.read_csv('./emails-word2vec.csv')

In [34]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

clf = LinearSVC(C=50)

X = eml_df[eml_df.columns[~eml_df.columns.isin(['SPAM_VALUE'])]].fillna(value=0)
y = eml_df['SPAM_VALUE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf.fit(X_train, y_train)

print(classification_report(clf.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.69      0.70      0.70        54
          1       0.96      0.96      0.96       415

avg / total       0.93      0.93      0.93       469

